## ValidMind Initialization


In [ ]:
import validmind as vm

vm.init(
  api_host = "...",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

## Prepare dataset 


In [ ]:
import pandas as pd

# strikes = [90, 95, 100, 105, 110]
# maturities = [0.5, 1.0, 1.5, 2.0]
data_df = pd.DataFrame({"K": [90, 95, 100, 105, 110], "T": [0.5, 1.0, 1.5, 2.0, 2.5]})
print(data_df)

vm_ds =  vm.init_dataset(
    data_df,
    input_id = "vm_dataset"
)

# BlackScholesModel

## model

In [4]:
import numpy as np
from scipy.optimize import minimize

class OptionPricing:
    def __init__(self, S0, r, M, N):
        self.S0 = S0
        self.r = r
        self.M = M 
        self.N = N 

    def monte_carlo_simulation(self, T):
        raise NotImplementedError("Must be implemented by subclasses")

    def price_option(self, K, T):
        raise NotImplementedError("Must be implemented by subclasses")


class BlackScholesModel(OptionPricing):
    def __init__(self, S0, r, M, N, sigma):
        super().__init__(S0, r, M, N)
        self.sigma = sigma

    def monte_carlo_simulation(self, T):
        dt = T / self.M
        price_paths = np.zeros((self.N, self.M + 1))
        price_paths[:, 0] = self.S0
        for t in range(1, self.M + 1):
            Z = np.random.standard_normal(self.N)
            price_paths[:, t] = price_paths[:, t - 1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * Z)
        return price_paths

    def price_option(self, K, T):
        price_paths = self.monte_carlo_simulation(T)
        payoffs = np.maximum(price_paths[:, -1] - K, 0)
        return np.exp(-self.r * T) * np.mean(payoffs)
    
    def calibrate(self, market_prices, strikes, maturities):
        def objective_function(params):
            self.sigma = params[0]
            model_prices = [self.price_option(K, T) for K, T in zip(strikes, maturities)]
            return np.sum((np.array(market_prices) - np.array(model_prices))**2)
        
        result = minimize(objective_function, [self.sigma], bounds=[(0.01, 1.0)])
        self.sigma = result.x[0]



##  Generate synthetic data


In [5]:
def generate_synthetic_market_data(model, strikes, maturities):
    market_prices = [model.price_option(K, T) for K, T in zip(strikes, maturities)]
    return market_prices

In [30]:
# Parameters for synthetic data
S0 = 100
r = 0.05
true_sigma = 0.2
true_v0 = 0.2
true_kappa = 2.0
true_theta = 0.2
true_xi = 0.1
true_rho = -0.5
N = 10000
M = 1000

strikes = [90, 95, 100, 105, 110]
maturities = [0.5, 1.0, 1.5, 2.0]

# Generate synthetic market data using the true parameters
bs_model = BlackScholesModel(S0, r, M, N, true_sigma)
bs_market_prices = generate_synthetic_market_data(bs_model, strikes, maturities)

## Calibrate models


In [ ]:
bs_model.calibrate(bs_market_prices, strikes, maturities)
print("Calibrated sigma:", bs_model.sigma)

## Prediction function for option price 

In [32]:

def predict_bs_option_price(input):
    price_paths = bs_model.monte_carlo_simulation(input["T"])
    payoffs = np.maximum(price_paths[:, -1] - input["K"], 0)
    return np.exp(-bs_model.r * input["T"]) * np.mean(payoffs)

vm_bs_model = vm.init_model(input_id="bs_model", predict_fn=predict_bs_option_price)

## Predict option price 

In [ ]:
vm_ds.assign_predictions(model=vm_bs_model)
print(vm_ds.df)

# Stochastic Volatility Model

## model

In [34]:
import numpy as np
from scipy.optimize import minimize

class OptionPricing:
    def __init__(self, S0, K, T, r):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r

    def monte_carlo_simulation(self, N, M):
        raise NotImplementedError("Must be implemented by subclasses")

    def price_option(self, N, M):
        raise NotImplementedError("Must be implemented by subclasses")


class StochasticVolatilityModel(OptionPricing):
    def __init__(self, S0, K, T, r, v0, kappa, theta, xi, rho):
        super().__init__(S0, K, T, r)
        self.v0 = v0
        self.kappa = kappa
        self.theta = theta
        self.xi = xi
        self.rho = rho

    def monte_carlo_simulation(self, N, M):
        dt = self.T / M
        price_paths = np.zeros((int(N), int(M) + 1))
        vol_paths = np.zeros((int(N), int(M) + 1))
        price_paths[:, 0] = self.S0
        vol_paths[:, 0] = self.v0
        for t in range(1, M + 1):
            Z1 = np.random.standard_normal(N)
            Z2 = np.random.standard_normal(N)
            W1 = Z1
            W2 = self.rho * Z1 + np.sqrt(1 - self.rho**2) * Z2
            vol_paths[:, t] = np.abs(vol_paths[:, t - 1] + self.kappa * (self.theta - vol_paths[:, t - 1]) * dt + self.xi * np.sqrt(vol_paths[:, t - 1] * dt) * W1)
            price_paths[:, t] = price_paths[:, t - 1] * np.exp((self.r - 0.5 * vol_paths[:, t - 1]) * dt + np.sqrt(vol_paths[:, t - 1] * dt) * W2)
        return price_paths

    def price_option(self, N, M):
        price_paths = self.monte_carlo_simulation(N, M)
        payoffs = np.maximum(price_paths[:, -1] - self.K, 0)
        return np.exp(-self.r * self.T) * np.mean(payoffs)
    
    def calibrate(self, market_prices, strikes, maturities):
        def objective_function(params):
            self.v0, self.kappa, self.theta, self.xi, self.rho = params
            model_prices = [self.price_option(10000, 100) for K, T in zip(strikes, maturities)]
            return np.sum((np.array(market_prices) - np.array(model_prices))**2)
        
        initial_guess = [self.v0, self.kappa, self.theta, self.xi, self.rho]
        bounds = [(0.01, 1.0), (0.01, 5.0), (0.01, 1.0), (0.01, 1.0), (-1.0, 1.0)]
        result = minimize(objective_function, initial_guess, bounds=bounds)
        self.v0, self.kappa, self.theta, self.xi, self.rho = result.x

##  Generate synthetic data


In [35]:
def generate_synthetic_market_data(model, strikes, maturities):
    market_prices = [model.price_option(10000, 100) for K, T in zip(strikes, maturities)]
    return market_prices

In [36]:
# Parameters for synthetic data
S0 = 100
r = 0.05
true_sigma = 0.2
true_v0 = 0.2
true_kappa = 2.0
true_theta = 0.2
true_xi = 0.1
true_rho = -0.5

strikes = [90, 95, 100, 105, 110]
maturities = [0.5, 1.0, 1.5, 2.0]

# Generate synthetic market data using the true parameters
sv_model = StochasticVolatilityModel(S0, 100, 1, r, true_v0, true_kappa, true_theta, true_xi, true_rho)
sv_market_prices = generate_synthetic_market_data(sv_model, strikes, maturities)


## Calibrate models


In [ ]:
sv_model.calibrate(sv_market_prices, strikes, maturities)
print("Calibrated parameters: v0 =", sv_model.v0, ", kappa =", sv_model.kappa, ", theta =", sv_model.theta, ", xi =", sv_model.xi, ", rho =", sv_model.rho)

## Prediction function for option price 

In [38]:

def predict_sv_option_price(input):
    price_paths = sv_model.monte_carlo_simulation(int(input["N"]), int(input["M"]))
    payoffs = np.maximum(price_paths[:, -1] - sv_model.K, 0)
    return np.exp(-sv_model.r * sv_model.T) * np.mean(payoffs)


vm_sv_model = vm.init_model(input_id="sv_model", predict_fn=predict_sv_option_price)

## Predict option price 

In [ ]:
import pandas as pd

# strikes = [90, 95, 100, 105, 110]
# maturities = [0.5, 1.0, 1.5, 2.0]
data_sv_df = pd.DataFrame({"K": [90, 95, 100, 105, 110], "T": [0.5, 1.0, 1.5, 2.0, 2.5]})
data_sv_df["N"] = 100000
data_sv_df["M"] = 100
print(data_sv_df)

vm_sv_ds =  vm.init_dataset(
    data_sv_df,
    input_id = "vm_sv_dataset"
)

In [ ]:
vm_sv_ds.assign_predictions(model=vm_sv_model)
print(vm_sv_ds)

# Knockout Option

# model

In [43]:
import matplotlib.pyplot as plt

class KnockoutOption:
    def __init__(self, model, S0, K, T, r, barrier):
        self.model = model
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.barrier = barrier


##  Generate synthetic data


In [44]:
# Example usage:
S0 = 100
T = 1
r = 0.05
N = 100000
M = 100
strike_range = (90, 110)
barrier_range = (100, 120)
sigma=0.2
bs_model = BlackScholesModel(S0, 100, 1, r, sigma=sigma)

# Generate synthetic market data using the true parameters
ko_model = KnockoutOption(bs_model, S0, strike_range[0], T, r, barrier_range[0])


## Prediction function for option price 

In [54]:
def predict_knockout_option_price(input):
        dt = ko_model.T / input["M"]
        price_paths = np.zeros((int(input["N"]), int(input["M"]) + 1))
        vol_paths = np.zeros((int(input["N"]), int(input["M"]) + 1)) if isinstance(ko_model.model, StochasticVolatilityModel) else None
        price_paths[:, 0] = ko_model.S0
        if vol_paths is not None:
            vol_paths[:, 0] = ko_model.model.v0
        
        for t in range(1, int(input["M"]) + 1):
            Z1 = np.random.standard_normal(int(input["N"]))
            if vol_paths is None:
                # Black-Scholes Model
                price_paths[:, t] = price_paths[:, t - 1] * np.exp(
                    (ko_model.r - 0.5 * ko_model.model.sigma**2) * dt + ko_model.model.sigma * np.sqrt(dt) * Z1
                )
            else:
                # Stochastic Volatility Model
                Z2 = np.random.standard_normal(int(input["N"]))
                W1 = Z1
                W2 = ko_model.model.rho * Z1 + np.sqrt(1 - ko_model.model.rho**2) * Z2
                vol_paths[:, t] = np.abs(vol_paths[:, t - 1] + ko_model.model.kappa * (ko_model.model.theta - vol_paths[:, t - 1]) * dt + ko_model.model.xi * np.sqrt(vol_paths[:, t - 1] * dt) * W1)
                price_paths[:, t] = price_paths[:, t - 1] * np.exp(
                    (ko_model.r - 0.5 * vol_paths[:, t - 1]) * dt + np.sqrt(vol_paths[:, t - 1] * dt) * W2
                )
            
            # Knockout condition
            price_paths[:, t][price_paths[:, t] >= ko_model.barrier] = 0
        
        payoffs = np.maximum(price_paths[:, -1] - ko_model.K, 0)
        return np.exp(-ko_model.r * ko_model.T) * np.mean(payoffs)


vm_ko_model = vm.init_model(input_id="ko_model", predict_fn=predict_knockout_option_price)

## Predict option price 

In [ ]:
import pandas as pd

# strikes = [90, 95, 100, 105, 110]
# maturities = [0.5, 1.0, 1.5, 2.0]
data_ko_df = pd.DataFrame({"K": [90, 95, 100, 105, 110], "T": [0.5, 1.0, 1.5, 2.0, 2.5]})
data_ko_df["N"] = 100000
data_ko_df["M"] = 100
print(data_ko_df)

vm_ko_ds =  vm.init_dataset(
    data_ko_df,
    input_id = "vm_ko_dataset"
)

In [ ]:
vm_ko_ds.assign_predictions(model=vm_ko_model)
print(vm_ko_ds)

In [ ]:
vm_ko_ds._df
